In [7]:
import torch
target = torch.randint(0, 10, (1000,), )
target  = 1 * (target == 9)
pred = torch.tensor(target)
pred[9:15] = 1 - pred[9:15]


/tmp/ipykernel_56591/1364576794.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred = torch.tensor(target)


In [9]:
from typing import Optional, Any
from torch import Tensor
from torchmetrics import IoU
from torchmetrics.functional.classification.iou import _iou_from_confmat

class BuildingsIoU(IoU):
    """ Custom IoU metrics to log building IoU only using PytorchLighting log system. """
    def __init__(
        self,
        compute_on_step: bool = True,
        dist_sync_on_step: bool = False,
        process_group: Optional[Any] = None,
        ) -> None:
        super().__init__(
            num_classes=2,
            threshold=0.5,
            reduction = "none",
            absent_score = 1.0,
            compute_on_step=compute_on_step,
            dist_sync_on_step=dist_sync_on_step,
            process_group=process_group,
        )
        

    def compute(self) -> Tensor:
        """Computes intersection over union (IoU)"""
        iou_no_reduction = _iou_from_confmat(self.confmat, self.num_classes, self.ignore_index, self.absent_score, self.reduction)
        iou_building = iou_no_reduction[1] 
        return iou_building

In [10]:
# ignore index functionnaly ignore the rows where class is 0 even if we predict 1 for them...
train_iou = BuildingsIoU()
train_iou(pred, target)

tensor(0.9412)

In [11]:
# ignore index functionnaly ignore the rows where class is 0 even if we predict 1 for them...
train_iou = IoU(2, reduction="none", absent_score=1.0)
train_iou(pred, target)

tensor([0.9934, 0.9412])

In [34]:
train_iou = IoU(2, reduction="elementwise_mean", absent_score=1.0)
train_iou(pred, target)

tensor(0.7846)

In [10]:
target.shape

torch.Size([10, 25, 25])